## Day 6

In [54]:
import numpy as np
import copy
from tqdm import tqdm
from itertools import product

with open("data_day6_grid.txt", 'r') as file:
    lines = [list(line.strip()) for line in file]

grid_init = np.array(lines, dtype = str)

def in_bounds(x, y, grid):
    rows, cols = grid.shape
    return 0 <= x < rows and 0 <= y < cols

def one_step(grid, pos, dir):
    i, j = pos
    a, b = dir
    grid[i][j] = "X"
    R = np.array([[0, 1], [-1, 0]])
    i_candidate, j_candidate = np.array([i,j]) + np.array([a,b])
    if not in_bounds(i_candidate, j_candidate, grid):
        pos_next = grid.shape
        dir_next = dir
    elif grid[i_candidate][j_candidate] == "#":
        pos_next = i, j
        dir_next = np.matmul(R, dir)
    else:
        pos_next = i_candidate, j_candidate
        dir_next = dir
    return grid, pos_next, dir_next       


In [55]:
m,n = grid_init.shape
for i, j in product(range(m),range(n)):
    if grid_init[i][j] == "^":
        i_init, j_init = i,j

pos_init = i_init, j_init
dir_init = np.array([-1,0])

grid = grid_init
pos = pos_init
dir = dir_init

while pos != grid.shape:
    grid, pos, dir = one_step(grid,pos,dir)

counter = 0
for i, j in product(range(m),range(n)):
    if grid[i][j] == "X":
        counter += 1

# task 1 output
print(counter)


4515


In [ ]:
import numpy as np
from tqdm import tqdm

# Define Rotation map
ROTATE_MAP = {(-1, 0): (0, 1), (0, 1): (1, 0), (1, 0): (0, -1), (0, -1): (-1, 0)}

def one_step_repeatcheck(pos, dir, history_set, loop_status, grid):
    i, j = pos
    a, b = dir

    m, n = grid.shape
    if not (0 <= i + a < m and 0 <= j + b < n):
        pos_next = (-1, -1)
        dir_next = dir
    elif grid[i + a, j + b] == "#":
        pos_next = (i, j)
        dir_next = np.array(ROTATE_MAP.get(tuple(dir), dir))
    else:
        pos_next = (i + a, j + b)
        dir_next = dir

    history_key = (tuple(pos_next), tuple(dir_next))
    if history_key in history_set:
        loop_status = True
    history_set.add(history_key)

    return pos_next, dir_next, history_set, loop_status


# Main loop execution
m, n = grid_init.shape
loop_counter = 0

# Find initial position
for i, j in product(range(m), range(n)):
    if grid_init[i][j] == "^":
        i_init, j_init = i, j

pos_init = (i_init, j_init)
dir_init = np.array([-1, 0])
grid = grid_init

loop_counter = 0
progress_bar = tqdm(product(range(m), range(n)), desc="Processing (i, j) pairs", total=m * n)

for i, j in progress_bar:
    obstacle_cell = grid[i, j]

    if (i, j) != (i_init, j_init):
        grid[i, j] = "#"

    pos = pos_init
    dir = dir_init
    history_set = set()
    loop_status = False

    while (0 <= pos[0] < m and 0 <= pos[1] < n) and (not loop_status):
        pos, dir, history_set, loop_status = one_step_repeatcheck(pos, dir, history_set, loop_status, grid)

    if loop_status:
        loop_counter += 1

    grid[i, j] = obstacle_cell  # Restore original grid state

print("Total loops detected:", loop_counter)


Processing (i, j) pairs: 100%|██████████| 16900/16900 [09:36<00:00, 29.29it/s] 

Total loops detected: 1309
